In [1]:
import gmsh
import matplotlib.pyplot as plt
plt.rc('figure',figsize=(10,10))
plt.rc('axes',labelsize=24)
plt.rc('xtick',labelsize=18)
plt.rc('ytick',labelsize=18)

import numpy as np

In [2]:
gmsh.initialize()

 aunque vallamos a poner los elementos a mano, necesitamos generar la superficie para asignarlos:

In [3]:
lc = 1
L = 10


p1 = gmsh.model.geo.addPoint(0, 0, 0, lc/5)
p2 = gmsh.model.geo.addPoint(2*L, 0, 0, lc) 
p3 = gmsh.model.geo.addPoint(2*L, L, 0, lc) 
p4 = gmsh.model.geo.addPoint(0, L, 0, lc/5)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])

S1 = gmsh.model.geo.addPlaneSurface([C1])

gmsh.model.geo.synchronize()

In [4]:
Empotrado = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1,Empotrado,'Empotrado')
Traccionado = gmsh.model.addPhysicalGroup(1, [l2])
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado')
Superficie = gmsh.model.addPhysicalGroup(2,[S1])
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')

In [5]:
L = 10
MN = np.array(
[
    [0,0,0],
    [2*L,0,0],
    [2*L,L,0 ],
    [0, L, 0]
]
)
nodetags = np.linspace(1,MN.shape[0], MN.shape[0]).astype(int)

In [6]:
MC = np.array([ [1,2,3], [1,3,4] ])
#MC = 

In [7]:
Empotrados = np.array([1,4])
Traccionados = np.array([2,3])

In [8]:
elementags = [[1,2]]

In [9]:
gmsh.model.mesh.add_nodes(2,S1,nodetags, MN.ravel(),)

In [10]:
gmsh.model.mesh.addElements(2, S1, [2], elementags, [MC.ravel()])

In [11]:
gmsh.model.mesh.add_elements(1,l4, [1], [[len(nodetags)+1]], [Empotrados])

In [12]:
gmsh.model.mesh.add_elements(1,l2, [1], [[len(nodetags)+2]], [Traccionados])

In [13]:
gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)


In [14]:
gmsh.write('test.msh')

In [15]:
gmsh.fltk.run()

In [16]:
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)

In [17]:
import mefmods as mef

In [18]:
MN

array([[ 0,  0,  0],
       [20,  0,  0],
       [20, 10,  0],
       [ 0, 10,  0]])

In [19]:
ETYPES = 2*np.ones(len(MC)).astype(int)

nu = 0.3

E = 210E9

MP = np.hstack(
        (
            np.ones((len(MC), 1)),
            np.ones((len(MC), 1))*nu,
            np.ones((len(MC), 1))*E
            )
        )

In [23]:
MC

array([[1, 2, 3],
       [1, 3, 4]])

In [25]:
K = mef.ensamble(MC-1, MN-1, MP, 2, ETYPES, 'Chapa-Symetrica-202')

In [16]:
gmsh.finalize()